In [10]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

import re
import xml.etree.ElementTree as ET
import pandas as pd

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *

pkg.name

'isr.umich.edu-psid-inequality-3.2.1'

In [12]:
varis = pkg.resource('variables').dataframe() # Can also do extract_varis(pkg)

In [13]:
# Build the schema just from the 1999 columns
def build(pkg, r, df ,  resource_name):
    # create URLs to the files inside the data distribution ZIP file


    # Recommended construction of unique person id
    df['pid'] = (df['ER30001'] * 1000) + df['ER30002']
    df['year'] = None
    df['race'] = None

    mdf = extract_metadata(varis, r)

    # Re-order the data dictionary
    r = pkg.resource(resource_name)
    mdf_sel = reorder_schema(r, mdf, df)  # Dataframe of mdf in new order for schema

    # HOWEVER ... for this dataset we only want the 1999 columns, since
    # the columns for the other years will be linked to the 1999 column
    mdf_sel = mdf_sel[mdf_sel.year.isin([1968,1999])]

    # Rebuild the package schema.

    new_terms = [
        {'value':'pid', 'DataType':'integer', 'description':'Constructed person id'},
        {'value':'year', 'DataType':'integer', 'description':'Year of the data'},
        {'value':'race', 'DataType':'text', 'description':'Extracted race of head of household'},
    ]

    return update_schema(r, mdf_sel, df, new_terms)


r = pkg.reference('long_wealth_source')
df = open_dbf(r)


build(pkg, r, df,  'long_wealth')

,pid,ER30001,ER30002,ER30000,ER33501,ER33503,ER13001,ER33502,ER13013,ER13010,...,ER16462,ER16518,S405,S419,S420,S415,S411,S413,S417,S416
0,4001,4,1,3,3,10,8.0,1,0.0,76.0,...,11718.0,31.473,2000.0,0.0,60000.0,0.0,0.0,4500.0,91500.0,31500.0
1,4003,4,3,3,2,10,8.0,1,1.0,47.0,...,62060.0,20.446,0.0,0.0,6000.0,0.0,8000.0,12000.0,26000.0,20000.0
2,4004,4,4,3,6129,10,8.0,1,0.0,45.0,...,35900.0,30.397,7500.0,17000.0,32000.0,0.0,7000.0,20000.0,413500.0,381500.0
3,4006,4,6,3,4920,22,8.0,2,0.0,41.0,...,81000.0,19.566,10000.0,0.0,20000.0,0.0,0.0,10000.0,40000.0,20000.0
4,4007,4,7,3,6944,10,8.0,1,1.0,39.0,...,35616.0,24.971,12000.0,0.0,15000.0,0.0,0.0,15000.0,42000.0,27000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17484,6872172,6872,172,3,2246,30,8.0,3,5.0,37.0,...,57468.0,4.323,10000.0,0.0,35000.0,500000.0,0.0,0.0,664000.0,629000.0
17485,6872174,6872,174,3,2246,30,8.0,5,5.0,37.0,...,57468.0,4.323,10000.0,0.0,35000.0,500000.0,0.0,0.0,664000.0,629000.0
17486,6872180,6872,180,3,2246,0,8.0,0,5.0,37.0,...,57468.0,4.323,10000.0,0.0,35000.0,500000.0,0.0,0.0,664000.0,629000.0
17487,6872182,6872,182,3,625,0,8.0,0,2.0,41.0,...,100938.0,3.555,12000.0,13000.0,80000.0,0.0,0.0,20000.0,123000.0,43000.0


In [14]:
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *

r = pkg.resource('long_wealth')
m = Metadata(r)

In [15]:
# Translate the 1999 columns to the columns for all of the other years,
# then slice them into a new dataframe, then concatenate all of the dataframes into the final dataset.

rows = []
years = [2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]

cols = list(r.columns())
for y in years:
    row = []
    for c in cols:
        col = m.translate_year(c['name'], y)
        try:
            row.append(col[0])
        except IndexError:
            row.append(None)

    rows.append(row)

lengths_per_year = [sum([c is not None for c in year_cols]) for year_cols in rows]

print("=== Vars per year ===")
for y, l in zip(years, lengths_per_year):
    #print(l, e['name'], e['label'])
    print(y, l)

print("=== Missing ===")

from collections import defaultdict

row = rows[-1]
missing = defaultdict(list)
all_vars = set()
for year, row in zip(years, rows):
    for cr, cy in zip(cols, row):
        if cy is None and cr['name'] not in ['pid', 'year'] and not cr['year'] == '1968':
            missing[cr['name']].append(year)
        else:
            all_vars.add(cy)

for k, v in missing.items():
    print(k, v)

=== Vars per year ===
2001 63
2003 63
2005 63
2007 63
2009 63
2011 63
2013 63
2015 63
2017 63
2019 62
=== Missing ===
race [2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]
s405 [2019]


In [16]:
# For each year, find the equivalent of the 1999 columns and slice
# them into a new dataframe, then concatenate all of the dataframes into the final dataset.

def xlate(resource, m, year):
    """Translate the 1999 columns to a new year"""
    cols = list(resource.columns())

    src = []
    dst = []
    for c in cols:
        cn = c['name']
        cy = c['year']

        if cy == '1968':
            src.append(cn.upper())
            dst.append(cn.upper())
        elif cy is None or cy == '':
            src.append(cn)
            dst.append(cn)
        else:
            col = m.translate_year(cn, year)

            try:
                src.append(col[0].upper())
                dst.append(cn.upper())
            except IndexError:
                pass

    return src, dst

pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *

r = pkg.resource('long_wealth')
m = Metadata(r)

frames = []
from tqdm.auto import tqdm

# Look for missing columns
extant_cols = set(df.columns.to_list())
missing = []

for year in tqdm(years, desc='Missing?'):
    src, dst = xlate(r, m, year)
    for c in src:
        if c not in extant_cols:
            missing.append(c)

if len(missing):
    print(" ".join([str(m) for m in missing]))
    raise KeyError("Missing columns")

# Actually translate the columns

for year in tqdm(years):
    src, dst = xlate(r, m, year)
    col_map = dict(zip(src, dst))

    yf = df.loc[:,src].assign(year=year).rename(columns=col_map)

    frames.append(yf)



dfl = pd.concat(frames) # df long

Missing?:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
# The race of each record will be the race of the head of household, which will be collapsed from
# multiple values over multiple years. Apparently they ask for race every year, so it can change from year to
# year, so we select the most common one
#
# L40 is 'race of head'

from pylib import race_merges

roh_cols = [c.upper() for c in m.search_label('L40', 1999).name.tolist() ]

print("Race Columns:", roh_cols)

races = {}
for gn, g in dfl.groupby('pid'):
    # Replace the values we aren't using, then convert the multiple race columns over multiple
    # years into a series. Take the most common value
    s = pd.Series(g[roh_cols].replace(race_merges).values.ravel()).value_counts()

    try:
        races[gn] = s.index[0]
    except IndexError:
        races[gn] = None


dfl['race'] = dfl.pid.apply(lambda pid: races[pid])

Race Columns: ['ER15928', 'ER15929', 'ER15930', 'ER15931']


In [18]:
# Ensure columns are in the same order as the schema
all_cols = [ c['name'].upper() if not c['name'] in ('race', 'pid','year') else c['name']
             for c in pkg.resource('long_wealth').columns() ]
longwealth = dfl[all_cols]

In [19]:

longwealth.shape

(174890, 68)